In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [1]:
data = pd.read_csv('output_merged_no_filter_5000.csv')
data = data.sort_values(by=['ID', 'YEAR', 'MONTH'])

NameError: name 'pd' is not defined

In [6]:
data = data[:5000]

In [7]:
data

,ID,YEAR,MONTH,VALUE,DMFLAG,QCFLAG,LATITUDE,LONGITUDE,STNELEV,NAME
0,ACW00011604,1961,1,-78,NaN,NaN,57.7667,11.8667,18.0,SAVE
36585,ACW00011604,1961,2,247,NaN,NaN,57.7667,11.8667,18.0,SAVE
73174,ACW00011604,1961,3,483,NaN,NaN,57.7667,11.8667,18.0,SAVE
93563,ACW00011604,1961,4,784,NaN,NaN,57.7667,11.8667,18.0,SAVE
128727,ACW00011604,1961,5,1139,NaN,NaN,57.7667,11.8667,18.0,SAVE
...,...,...,...,...,...,...,...,...,...,...
8911,ASN00014507,1961,1,2893,E,NaN,-13.8483,136.4198,20.0,ALYANGULA_POLICE
36044,ASN00014507,1961,2,2925,E,NaN,-13.8483,136.4198,20.0,ALYANGULA_POLICE
71587,ASN00014507,1961,3,2871,E,NaN,-13.8483,136.4198,20.0,ALYANGULA_POLICE
97909,ASN00014507,1961,4,2748,E,NaN,-13.8483,136.4198,20.0,ALYANGULA_POLICE


In [8]:
# 1. Scale numerical features
scaler = MinMaxScaler()
numerical_features = ['LATITUDE', 'LONGITUDE', 'STNELEV']
data[numerical_features] = scaler.fit_transform(data[numerical_features])

In [9]:
data

,ID,YEAR,MONTH,VALUE,DMFLAG,QCFLAG,LATITUDE,LONGITUDE,STNELEV,NAME
0,ACW00011604,1961,1,-78,NaN,NaN,1.000000,0.553626,0.003992,SAVE
36585,ACW00011604,1961,2,247,NaN,NaN,1.000000,0.553626,0.003992,SAVE
73174,ACW00011604,1961,3,483,NaN,NaN,1.000000,0.553626,0.003992,SAVE
93563,ACW00011604,1961,4,784,NaN,NaN,1.000000,0.553626,0.003992,SAVE
128727,ACW00011604,1961,5,1139,NaN,NaN,1.000000,0.553626,0.003992,SAVE
...,...,...,...,...,...,...,...,...,...,...
8911,ASN00014507,1961,1,2893,E,NaN,0.364042,0.931293,0.004191,ALYANGULA_POLICE
36044,ASN00014507,1961,2,2925,E,NaN,0.364042,0.931293,0.004191,ALYANGULA_POLICE
71587,ASN00014507,1961,3,2871,E,NaN,0.364042,0.931293,0.004191,ALYANGULA_POLICE
97909,ASN00014507,1961,4,2748,E,NaN,0.364042,0.931293,0.004191,ALYANGULA_POLICE


In [10]:
# 2. One-hot encode categorical features
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)  # sparse=False for array output
categorical_features = ['ID', 'DMFLAG', 'QCFLAG', 'NAME']  # Include 'ID' for station embeddings
encoded_features = encoder.fit_transform(data[categorical_features])

In [11]:
X = np.concatenate([encoded_features,
                   data[numerical_features].values], axis=1)
y = data['VALUE'].values  # Target variable

In [19]:
num_features = X[1].shape[0]

In [20]:
num_features

1116

In [21]:
# --- Sequence creation ---
sequence_length = 12  # Example sequence length (12 months)

X_sequences = []
y_sequences = []
for i in range(sequence_length, len(X)):
    X_sequences.append(X[i - sequence_length:i])
    y_sequences.append(y[i])

X_sequences = np.array(X_sequences)
y_sequences = np.array(y_sequences)

In [22]:
# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X_sequences, y_sequences, test_size=0.2, random_state=42
)

# Further split training set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split( X_train, y_train, test_size=0.1, random_state=42
)

In [23]:
# Define the transformer model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(sequence_length, num_features)),
    tf.keras.layers.Transformer(
        num_layers=num_layers,
        d_model=d_model,
        num_heads=num_heads,
        dff=dff
    ),
    tf.keras.layers.Dense(1) # Output layer for temperature prediction
])

AttributeError: module 'keras._tf_keras.keras.layers' has no attribute 'Transformer'

In [ ]:
# Compile the model
model.compile(loss='mse', optimizer='adam')

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=epochs, validation_data=(X_val, y_val))